In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# #check file paths for data paths 
# from os import listdir

# #list files/directories in current directory
# for file in listdir("./"):
#     print(file)
    
train_path = "./data/train.csv"
test_path = "./data/test.csv"
test_labels_path = "./data/test_labels.csv"

#load data with pandas and return dataframes 
def load_data(train_path, test_path, test_labels_path):
    train = pd.read_csv(train_path)
    print(train[0:10])
    test = pd.read_csv(test_path)
    test_labels = pd.read_csv(test_labels_path)
    cols = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']
    return train, test, test_labels, cols


train, test, test_labels, cols = load_data(train_path, test_path, test_labels_path)

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   
5  00025465d4725e87  "\n\nCongratulations from me as well, use the ...      0   
6  0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1   
7  00031b1e95af7921  Your vandalism to the Matt Shirvington article...      0   
8  00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...      0   
9  00040093b2687caa  alignment on this subject and which are contra...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0          

In [3]:
#summarize data
def summary():
    labels = train.iloc[:,2:]
    zero = np.where(np.sum(labels,axis=1)==0)
    # Find the unlabelled percentage
    unlabelled = train[(train['toxic']!=1) & (train['severe_toxic']!=1) &\
                                 (train['obscene']!=1) & (train['threat']!=1) &\
                                 (train['insult']!=1) & (train['identity_hate']!=1)]
    print("Train data length: ", len(train))
    print("Test data length: ", len(test))
    print('\nPercentage of unlabelled: ', len(unlabelled)/len(train)*100)
    print(train[cols].sum())

    train[cols].sum().plot.bar(title ="Number of Comments by Label")


In [4]:

def get_data(num_to_use): 
    #temporarily only use 500 
    train_comments = train.iloc[0:num_to_use,1]
    test_comments = test.iloc[0:num_to_use,1]
    
    return test_comments, train_comments

test_comments, train_comments= get_data(len(train))


# each label as a single combined string, e.g. toxic & severe_toxic = 11
def get_labels_as_vec(num_to_use):
    #labels as numpy 
    labels = train.iloc[0:num_to_use,2:]
    labels['vector_label'] = labels.astype(str).values.sum(axis=1)
    train_labels = labels['vector_label']
    
    return train_labels

In [ ]:
#alternate : each label as a separate pd column
def get_labels_as_cols(num_to_use):
    #labels as pd dataframes
    toxic_train_labels = train.iloc[0:num_to_use,2:3]
    severe_toxic_labels = train.iloc[0:num_to_use,3:4] 
    obscene_labels = train.iloc[0:num_to_use,4:5] 
    threat_labels = train.iloc[0:num_to_use,5:6] 
    insult_labels = train.iloc[0:num_to_use,6:7] 
    identity_hate_labels = train.iloc[0:num_to_use,7:8] 
    
    return toxic_train_labels, severe_toxic_labels, obscene_labels, threat_labels, insult_labels, identity_hate_labels

toxic_train_labels, severe_toxic_labels, obscene_labels, threat_labels, insult_labels, identity_hate_labels = get_labels_as_cols(len(train))


In [ ]:
#vectorize inputs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


def vectorize_tfidf():
    vectorizer = TfidfVectorizer(max_df=0.5, stop_words='english', ngram_range=(1,2))
    features_train = vectorizer.fit_transform(train_comments)
    features_test = vectorizer.fit_transform(test_comments)
    train_vocab = vectorizer.get_feature_names()
#     print(train_vocab)
    return features_train, features_test, vectorizer

def vectorize_count():
    vectorizer = CountVectorizer(max_df=0.5, stop_words='english', ngram_range=(1, 2))
    features_train = vectorizer.fit_transform(train_comments)
    features_test = vectorizer.fit_transform(test_comments)
    train_vocab = vectorizer.get_feature_names()
#     print(train_vocab)
    return features_train, features_test, vectorizer

features_train, features_test, vectorizer = vectorize_tfidf()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, recall_score, precision_score, accuracy_score
from sklearn.model_selection import cross_validate

#Multinomial NB
def train_model(labels):
    model = MultinomialNB()
    model.fit(features_train, labels)
    score = model.score(features_train, labels)
    return score


def get_avg_cvd_score(model, X, y):
    to_get = { 'accuracy' : make_scorer(accuracy_score) }#,
   # 'precision' : make_scorer(precision_score, average='macro'),
    #'recall' : make_scorer(recall_score, average='macro')}
    results = cross_validate(model, X, y, scoring=to_get, cv=5)

    return np.average(results['test_accuracy'])#, np.average(results['test_precision']), np.average(results['test_recall'])


In [ ]:
#toxic
score_toxic = train_model(toxic_train_labels.values.ravel())
toxic_accuracy = get_avg_cvd_score(MultinomialNB(), features_train, toxic_train_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, toxic_train_labels.values.ravel())
print("toxic")
print("training accuracy: " + str(score_toxic))
print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
print("\n")

#severe_toxic_labels
print("severe_toxic")
score_severe_toxic = train_model(severe_toxic_labels.values.ravel())
toxic_accuracy =  get_avg_cvd_score(MultinomialNB(), features_train, severe_toxic_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, severe_toxic_labels.values.ravel())
print(score_severe_toxic)
print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
print("\n")

#obscene_labels
print("obscene")
score_obscene = train_model(obscene_labels.values.ravel())
toxic_accuracy =  get_avg_cvd_score(MultinomialNB(), features_train, obscene_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, obscene_labels.values.ravel())
print(score_obscene)
print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
print("\n")

#threat_labels
print("threat")
score_threat = train_model(threat_labels.values.ravel())
toxic_accuracy = get_avg_cvd_score(MultinomialNB(), features_train, threat_labels.values.ravel())
#toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, threat_labels.values.ravel())
print(score_threat)
print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
print("\n")

#insult_labels
print("insult")
score_insult = train_model(insult_labels.values.ravel())
toxic_accuracy = get_avg_cvd_score(MultinomialNB(), features_train, insult_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, insult_labels.values.ravel())
print(score_insult)
print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
print("\n")

#identity_hate_labels
print("identity_hate")
score_identity_hate = train_model(identity_hate_labels.values.ravel())
toxic_accuracy = get_avg_cvd_score(MultinomialNB(), features_train, identity_hate_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, identity_hate_labels.values.ravel())
print(score_identity_hate)
print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import make_scorer, recall_score, precision_score, accuracy_score
from sklearn.model_selection import cross_validate

#Multinomial NB
def train_model(labels):
    model = GaussianNB()
    model.fit(features_train, labels)
    score = model.score(features_train, labels)
    return score


def get_avg_cvd_score(model, X, y):
    to_get = { 'accuracy' : make_scorer(accuracy_score),
    'precision' : make_scorer(precision_score, average='macro'),
    'recall' : make_scorer(recall_score, average='macro')}
    results = cross_validate(model, X, y, scoring=to_get, cv=5)

    return np.average(results['test_accuracy']), np.average(results['test_precision']), np.average(results['test_recall'])

#change to array
features_train = features_train.toarray()
#toxic
score = train_model(toxic_train_labels.values.ravel())
toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, toxic_train_labels.values.ravel())
print("toxic")
print("training accuracy: " + str(score_toxic))
print("validation accuracy: " + str(toxic_accuracy))
print("precision: " +str(toxic_precision))
print("recall: " +str(toxic_recall))
# print("\n")

# #severe_toxic_labels
# print("severe_toxic")
# score_severe_toxic = train_model(severe_toxic_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, severe_toxic_labels.values.ravel())
# print(score_severe_toxic)
# print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
# print("\n")

# #obscene_labels
# print("obscene")
# score_obscene = train_model(obscene_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, obscene_labels.values.ravel())
# print(score_obscene)
# print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
# print("\n")

# #threat_labels
# print("threat")
# score_threat = train_model(threat_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, threat_labels.values.ravel())
# print(score_threat)
# print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
# print("\n")

# #insult_labels
# print("insult")
# score_insult = train_model(insult_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, insult_labels.values.ravel())
# print(score_insult)
# print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))
# print("\n")

# #identity_hate_labels
# print("identity_hate")
# score_identity_hate = train_model(identity_hate_labels.values.ravel())
# toxic_accuracy, toxic_precision, toxic_recall = get_avg_cvd_score(MultinomialNB(), features_train, identity_hate_labels.values.ravel())
# print(score_identity_hate)
# print("validation accuracy: " + str(toxic_accuracy))
# print("precision: " +str(toxic_precision))
# print("recall: " +str(toxic_recall))